# Async postgres libraries research

- Multyprocessing in Python 
- Why do we need to use asyncio?
- Can we still use pg8000?
- Libraries

# Multyprocessing in Python 


## Sync vs Async
- In Syncrhonous operations, the tasks are executed in sync, one after one. 
- In asynchronous operations, tasks may start and complete independent of each other. One async task may start and continue running while the execution moves on to a new task. 
- Async tasks don’t block (make the execution wait for it’s completion) operations and usually run in the background.

## Concurrency and Parallelism
- Concurrency implies that two tasks make progress together. 
- Parallelism is in fact a form of concurrency. 
- But parallelism is hardware dependent. 
- For example if there’s only one core in the CPU, two operations can’t really run in parallel. 

## Summary:
* Sync: Blocking operations.
* Async: Non blocking operations.
* Concurrency: Making progress together.
* Parallelism: Making progress in parallel.

Email, phone call example

### Parallelism implies Concurrency. But Concurrency doesn’t always mean Parallelism.

## Threads & Processes

# Threads
- Python has had Threads for a very long time. 
- Threads allow us to run our operations concurrently. 
- There is a problem with the Global Interpreter Lock (GIL) for which the **threading could not provide true parallelism.**

# Global Interpreter Lock (GIL)
The Global Interpreter Lock aka GIL was introduced to make CPython’s memory handling easier and to allow better integrations with C (for example the extensions). The GIL is a locking mechanism that the Python interpreter runs only one thread at a time. That is only one thread can execute Python byte code at any given time. This GIL makes sure that multiple threads DO NOT run in parallel.

Quick facts about the GIL:

* One thread can run at a time.
* The Python Interpreter switches between threads to allow concurrency.
* The GIL is only applicable to CPython (the defacto implementation). Other implementations like Jython, IronPython don’t have GIL.
* GIL makes single threaded programs fast.
* For I/O bound operations, GIL usually doesn’t harm much.
* GIL makes it easy to integrate non thread safe C libraries, thansk to the GIL, we have many high performance extensions/modules written in C.
* For CPU bound tasks, the interpreter checks between N ticks and switches threads. So one thread does not block others.

## Processes
- To get parallelism, Python introduced the multiprocessing module.
- Every core can take one process



# Why don't we use Threads to solve our issue?
* Processes are costly to spawn. 
* We know that I/O depends on external stuff - slow disks or nasty network lags make I/O often unpredictable. 
* 3 threads example. 

# 3 threads example.
Processes are costly to spawn. So for I/O, Threads are chosen largely. We know that I/O depends on external stuff - slow disks or nasty network lags make I/O often unpredictable. Now, let’s assume that we are using threads for I/O bound operations. 3 threads are doing different I/O tasks. The interpreter would need to switch between the concurrent threads and give each of them some time in turns. Let’s call the threads - T1, T2 and T3. The three threads have started their I/O operation. T3 completes it first. T2 and T1 are still waiting for I/O. The Python interpreter switches to T1 but it’s still waiting. Fine, so it moves to T2, it’s still waiting and then it moves to T3 which is ready and executes the code. Do you see the problem here?

T3 was ready but the interpreter switched between T2 and T1 first - that incurred switching costs which we could have avoided if the interpreter first moved to T3, right?

http://masnun.rocks/2016/10/06/async-python-the-different-forms-of-concurrency/

# Why do we need to use asyncio?
- Asyncio provides us an event loop. 
- The event loop tracks different I/O events and **switches to tasks which are ready** and **pauses the ones which are waiting on I/O**.
- Thus we don’t waste time on tasks which are not ready to run right now.
- Multithreading programming is all about concurrent execution of different functions. 
- Async programming is about non-blocking execution between functions

The idea is very simple. 
- There’s an event loop. 
- And we have functions that run async, I/O operations. 
- We give our functions to the event loop and ask it to run those for us. 
- The event loop gives us back a Future object, it’s like a promise that we will get something back in the future. 
- We hold on to the promise, time to time check if it has a value (when we feel impatient) and finally when the future has a value, we use it in some other operations.

## Why do we need to use asyncio?
* CPU Bound => Multi Processing
* I/O Bound, Fast I/O, Limited Number of Connections => Multi Threading
* I/O Bound, Slow I/O, Many connections => Asyncio

- It is very difficult to write code that is thread safe. With asyncronous code, you know exactly where the code will shift from one task to the next and race conditions are therefore much harder to come by.
- Threads consume a fair amount of data since each thread needs to have its own stack. With async code, all the code shares the same stack and the stack is kept small due to continuously unwinding the stack between tasks.
- Threads are OS structures and are therefore more memory for the platform to support. There is no such problem with asynchronous tasks.

# Can we still use pg8000?
Yes, if we use threads. But it's not the best solution in our case, since we wan't to have a lot of calls to db and a lot of connections.

# Async postgres libraries

We will compare 3 libraries today:
1. aiopg
2. psycopg3
3. asyncpg 

In [1]:
# Postgres database details:
host = "host"
db_name = "db_name"
port = "5432"
username = "username"
password = "password"
schema = "schema"

In [2]:
PINK = '\033[38;5;205m'
TEAL = '\033[38;5;31m'
GREEN = '\033[32m'
RESET = '\033[0m'

def print_pink(msg):
    print(PINK + str(msg) + RESET)
    
def print_teal(msg):
    print(TEAL + str(msg) + RESET)
    
def print_green(msg):
    print(GREEN + str(msg) + RESET)

    
def print_in_color(msg, color):
    if color == 'green':
        print_green(msg)
    elif color == 'teal':
        print_teal(msg)
    else:
        print_pink(msg)

# aiopg

- Python: 3.6+ 
- OS: Linux, macOS, Windows
- Uses psycopg2-binary and requires sqlalchemy

- 1327 stars
- last release October 2022 (9 month ago)
- 59 issues
- 12 open PRs

In [3]:
import sqlalchemy
import asyncio
from aiopg.sa import create_engine as aiopg_create_engine
from IPython.core.display import Image, display

async def aiopg_async_get_monitors_many_calls(color="green"):
    # Open async connection
    print_in_color(f"Openning connection", color=color)
    async with aiopg_create_engine(
        host=host,
        port=port,
        user=username,
        password=password,
        dbname=db_name,
        options=f"-c search_path=dbo,{schema}"
    ) as engine:
        
        async with engine.acquire() as conn:
            async for row in conn.execute(f"""SELECT id, name FROM dh_monitors LIMIT 3"""):
                print_in_color(row, color)

/var/folders/9z/rhfc541d14q5klxp9lqvgx8m0000gq/T/ipykernel_23983/2917714263.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import Image, display


In [4]:
asyncio.gather(
    aiopg_async_get_monitors_many_calls(color='green'),
    aiopg_async_get_monitors_many_calls(color="pink"),
    aiopg_async_get_monitors_many_calls(color="teal")
)

<_GatheringFuture pending>

Openning connection
Openning connection
Openning connection
(UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'monitor_for_deletion')
(UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'vitaly_leen_reg_mon_1')
(UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'not_standalone_monitor_007')
(UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'monitor_for_deletion')
(UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'vitaly_leen_reg_mon_1')
(UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'not_standalone_monitor_007')
(UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'monitor_for_deletion')
(UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'vitaly_leen_reg_mon_1')
(UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'not_standalone_monitor_007')


# psycopg3

- Python: from version 3.7 to 3.11
- Python 3.6 supported before Psycopg 3.1
- PostgreSQL: from version 10 to 15 (We use 14)
- OS: Linux, macOS, Windows

- 1100 stars
- last commit recently
- 35 issues
- 11 open PRs

### Sync way

In [5]:
import psycopg
from psycopg.rows import dict_row

def sync_get_monitors(color):
    with psycopg.connect(
        host=host,
        port=port,
        user=username,
        password=password,
        dbname=db_name,
        options=f"-c search_path=dbo,{schema}"
    ) as conn:

        # Open a cursor to perform database operations
        with conn.cursor(row_factory=dict_row) as cur:
            cur.execute("""SELECT id, name FROM dh_monitors LIMIT 3""")
            for record in cur:
                print_in_color(record, color=color)

sync_get_monitors(color="teal")

{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}


### Async way

In [6]:
async def psycopg3_async_get_monitors_many_calls(color="green"):
    # Open async connection
    print_in_color(f"Openning connection", color=color)
    async with await psycopg.AsyncConnection.connect(
        host=host,
        port=port,
        user=username,
        password=password,
        dbname=db_name,
        options=f"-c search_path=dbo,{schema}") as aconn:
        
        # Open an async cursor to perform database operations
        async with aconn.cursor(row_factory=dict_row) as acur:
            await acur.execute(f"""SELECT id, name FROM dh_monitors LIMIT 3""")
            async for record in acur:
                print_in_color(record, color=color)
            print_in_color(f"fetched 100 %", color=color)

In [7]:
asyncio.gather(
    psycopg3_async_get_monitors_many_calls(color='green'),
    psycopg3_async_get_monitors_many_calls(color="pink"),
    psycopg3_async_get_monitors_many_calls(color="teal")
)

<_GatheringFuture pending>

Openning connection
Openning connection
Openning connection
{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}
fetched 100 %
{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}
fetched 100 %
{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}
fetched 100 %


### Async way with one connection

In [8]:
async def async_get_monitors_with_connection(aconn, color="green"):
    print_in_color(f"Creating cursor", color=color)            
    acur = aconn.cursor(row_factory=dict_row)
    
    # Open an async cursor to perform database operations
    print_in_color(f"Starting fetching", color=color)            
    await acur.execute(f"""SELECT id, name FROM dh_monitors LIMIT 3""")
    async for record in acur:
        print_in_color(record, color=color)
    print_in_color(f"fetched 100 %", color=color)

In [9]:
print(f"Openning connection")     
aconn = await psycopg.AsyncConnection.connect(
    host=host,
    port=port,
    user=username,
    password=password,
    dbname=db_name,
    options=f"-c search_path=dbo,{schema}")

asyncio.gather(
    async_get_monitors_with_connection(aconn=aconn, color='green'),
    async_get_monitors_with_connection(aconn=aconn, color="pink"),
    async_get_monitors_with_connection(aconn=aconn, color="teal")
)

Openning connection


<_GatheringFuture pending>

Creating cursor
Starting fetching
Creating cursor
Starting fetching
Creating cursor
Starting fetching
{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}
fetched 100 %
{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}
fetched 100 %
{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}
fetched 100 %


### Async way with one cursor

In [10]:
async def async_get_monitors_with_cursor(acur, color="green"):
    # Open an async cursor to perform database operations
    print_in_color(f"Starting fetching", color=color)            
    await acur.execute(f"""SELECT id, name FROM dh_monitors LIMIT 3""")
    async for record in acur:
        print_in_color(record, color=color)
    print_in_color(f"fetched 100 %", color=color)

In [11]:
print(f"Openning connection")     
aconn = await psycopg.AsyncConnection.connect(
    host=host,
    port=port,
    user=username,
    password=password,
    dbname=db_name,
    options=f"-c search_path=dbo,{schema}")
acur = aconn.cursor(row_factory=dict_row)

asyncio.gather(
    async_get_monitors_with_cursor(acur=acur, color='green'),
    async_get_monitors_with_cursor(acur=acur, color="pink"),
    async_get_monitors_with_cursor(acur=acur, color="teal")
)

Openning connection


<_GatheringFuture pending>

Starting fetching
Starting fetching
Starting fetching
{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}
fetched 100 %
{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}
fetched 100 %
{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}
fetched 100 %


### Async way with one cursor and await

In [12]:
async def async_get_monitors_with_cursor_and_await(acur, color="green"):
    # Open an async cursor to perform database operations
    print_in_color(f"Starting fetching", color=color)            
    await acur.execute(f"""SELECT id, name FROM dh_monitors LIMIT 3""")
    for i in range(3):
        print_in_color(await acur.fetchone(), color=color)
        await asyncio.sleep(0)
    print_in_color(f"fetched 100 %", color=color)


In [13]:
print(f"Openning connection")     
aconn = await psycopg.AsyncConnection.connect(
    host=host,
    port=port,
    user=username,
    password=password,
    dbname=db_name,
    options=f"-c search_path=dbo,{schema}")
acur = aconn.cursor(row_factory=dict_row)


asyncio.gather(
    async_get_monitors_with_cursor(acur=acur, color='green'),
    async_get_monitors_with_cursor(acur=acur, color="pink"),
    async_get_monitors_with_cursor(acur=acur, color="teal")
)

Openning connection


<_GatheringFuture pending>

Starting fetching
Starting fetching
Starting fetching
{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}
fetched 100 %
{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}
fetched 100 %
{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}
fetched 100 %


# asyncpg

- Python: 3.7 +
- PostgreSQL: from version 9.5- to 15 (We use 14)

- 6100 stars
- last commit recently
- 181 issues
- 20 open PRs

In [14]:
import asyncpg
async def asyncpg_async_get_monitors_many_calls(color="green"):    
    print_in_color(f"Openning connection", color=color)
    conn = await asyncpg.connect(
        host=host,
        port=port,
        user=username,
        password=password,
        database=db_name,
        server_settings={'search_path': schema}
        )
    rows = await conn.fetch(f"""SELECT id, name FROM dh_monitors LIMIT 3""")
    for row in rows:
        print_in_color(dict(row), color=color)

    await conn.close()

In [15]:
asyncio.gather(
    asyncpg_async_get_monitors_many_calls(color='green'),
    asyncpg_async_get_monitors_many_calls(color="pink"),
    asyncpg_async_get_monitors_many_calls(color="teal")
)

<_GatheringFuture pending>

Openning connection
Openning connection
Openning connection
{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}
{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}
{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}


### Async way with functions that require many db calls

In [16]:
import asyncio
async def async_get_monitors_many_calls(color="green"):
    # Open async connection
    print_in_color(f"Openning connection", color=color)
    async with await psycopg.AsyncConnection.connect(
        host=host,
        port=port,
        user=username,
        password=password,
        dbname=db_name,
        options=f"-c search_path=dbo,{schema}") as aconn:
        
        # Open an async cursor to perform database operations
        async with aconn.cursor(row_factory=dict_row) as acur:
            await acur.execute(f"""SELECT id, name FROM dh_monitors LIMIT 3""")
            async for record in acur:
                print_in_color(record, color=color)
                await asyncio.sleep(0)
            print_in_color(f"fetched 100 %", color=color)

In [17]:
asyncio.gather(
    async_get_monitors_many_calls(color='green'),
    async_get_monitors_many_calls(color="pink"),
    async_get_monitors_many_calls(color="teal")
)

<_GatheringFuture pending>

Openning connection
Openning connection
Openning connection
{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}
fetched 100 %
{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
{'id': UUID('c17feb88-db0b-4c58-84c1-8bac48b0dc77'), 'name': 'monitor_for_deletion'}
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}
{'id': UUID('e72bd6e6-b880-4b64-b91b-dedfa98fb12e'), 'name': 'vitaly_leen_reg_mon_1'}
fetched 100 %
{'id': UUID('6f2b03f0-bedb-4b31-b937-190d88c1f040'), 'name': 'not_standalone_monitor_007'}
fetched 100 %


# Compare 

## Girhub data
* aiopg - 1327, last release October 2022
* psycopg3 - 1100, last commit recently
* asyncpg - 6100, last commit recently

## Benchmark data (from asyncpg creators research)


In [9]:
display(Image(url='img/Benchmark.png', width=800, unconfined=True))

## Benchmark data
The above results are a geometric mean of benchmarks obtained with PostgreSQL client driver benchmarking toolbench in June 2023 (click on the chart to see full details).


The above results are a geometric mean of benchmarks obtained with PostgreSQL client driver benchmarking toolbench in June 2023 (click on the chart to see full details).  [Link to the full research](https://gistpreview.github.io/?0ed296e93523831ea0918d42dd1258c2)   
**However, the [research](https://github.com/MagicStack/asyncpg?tab=readme-ov-file) was done by asyncpg creators, so please be caution with using this benchmark.**

## In general

- aiopg:

- psycopg3: Extensive docs, docs on async topic include useful examples.In one of the PRs, a main contributor shared a blogpost (May 2020) they wrote. The topic was optimization and tagged with psycopg and development. asyncpg is faster, but psycopg aims to be more user-friendly

- asyncpg: extensive README. Includes benchmark showing it's supposedly 3x faster than aiopg and psycopg2, psycopg3 is not mentioned in the benchmark.


# aiopg

- Python: 3.6+ 
- OS: Linux, macOS, Windows
- Uses psycopg2-binary and requires sqlalchemy

- 1327 stars
- last release October 2022 (9 month ago)
- 59 issues
- 12 open PRs

# psycopg3

- Python: from version 3.7 to 3.11
- Python 3.6 supported before Psycopg 3.1
- PostgreSQL: from version 10 to 15 (We use 14)
- OS: Linux, macOS, Windows

- 1100 stars
- last commit recently
- 35 issues
- 11 open PRs

# asyncpg

- Python: 3.7 +
- PostgreSQL: from version 9.5- to 15 (We use 14)

- 6100 stars
- last commit recently
- 181 issues
- 20 open PRs

### For future read:
- https://www.varrazzo.com/blog/2020/05/19/a-trip-into-optimisation/

# PRESENTATION IS OVER.